# Objective :-

Here we have used an autoencoder like artitecture to create more features take for example there is a input data X and a target y .  So the neural network will try to map a linear function it will take input X , and also the target will be X or the input itself something like f(x)=x where f(x) is the model a particular hidden layer of the network will generate a 64 dimensional output which is our 64 new features You can find the model or the output file here :- https://www.kaggle.com/swaralipibose/autoencoderfeat it is a keras ann and is stored as'model {asset_id}.h5' format the model has also been trained with some extra features in ```get_features``` function so basically model.predict(get_features(inputs)).shape=(n,64). If you want the new features generated for ```train.csv``` [ ```test.csv``` is not provided and you may need to compute that using the models] they are stored .npy format '{asset_id}.npy' is the name you can find it here https://www.kaggle.com/swaralipibose/autoencoderfeatures the notebook covers the training of the autoencoder :-)

In [ ]:
import pandas as pd
import numpy as np
from lightgbm import LGBMRegressor
import gresearch_crypto


TRAIN_CSV = '/kaggle/input/g-research-crypto-forecasting/train.csv'
ASSET_DETAILS_CSV = '/kaggle/input/g-research-crypto-forecasting/asset_details.csv'

In [ ]:
df_train = pd.read_csv(TRAIN_CSV)
df_train.head()

In [ ]:
df_asset_details = pd.read_csv(ASSET_DETAILS_CSV).sort_values("Asset_ID")
df_asset_details

# Training

## Utility functions to train a model for one asset

## Loop over all assets

In [ ]:
from scipy.stats import pearsonr
from sklearn.model_selection import train_test_split
from keras.layers import *
from keras import *
from tensorflow.keras.optimizers import *
# Two new features from the competition tutorial
def upper_shadow(df):
    return df['High'] - np.maximum(df['Close'], df['Open'])

def lower_shadow(df):
    return np.minimum(df['Close'], df['Open']) - df['Low']

# A utility function to build features from the original df
# It works for rows to, so we can reutilize it.
def get_features(df):
    df_feat = df[['Count', 'Open', 'High', 'Low', 'Close', 'Volume', 'VWAP']].copy()
    df_feat['upper_Shadow'] = upper_shadow(df_feat)
    df_feat['lower_Shadow'] = lower_shadow(df_feat)
    df_feat["high_div_low"] = df_feat["High"] / df_feat["Low"]
    #df_feat["open_sub_close"] = df_feat["Open"] - df_feat["Close"]
    df_feat['trade']=df_feat['Close']-df_feat['Open']
    df_feat['gtrade']=df_feat['trade']/df_feat['Count']
    df_feat['shadow1']=df_feat['trade']/df_feat['Volume']
    #df_feat['shadow2']=df_feat['upper_Shadow']/df['Low']
    df_feat['shadow3']=df_feat['upper_Shadow']/df_feat['Volume']
    #df_feat['shadow4']=df_feat['lower_Shadow']/df['High']
    df_feat['shadow5']=df_feat['lower_Shadow']/df_feat['Volume']
    
    df_feat['diff1']=df_feat['Volume']-df_feat['Count']
    
    df_feat['mean1']=(df_feat['shadow5']+df_feat['shadow3'])/2
    
    df_feat['mean2']=(df_feat['shadow1']+df_feat['Volume'])/2
    
    df_feat['mean3']=(df_feat['trade']+df_feat['gtrade'])/2
    
    df_feat['mean4']=(df_feat['diff1']+df_feat['upper_Shadow'])/2
    
    df_feat['mean5']=(df_feat['diff1']+df_feat['lower_Shadow'])/2 
    
    return df_feat
def log(model,X_train, X_valid, y_train, y_valid,train_split=1.0):
    if train_split > 0:
        X_train=X_train[:int(train_split*X_train.shape[0])]
        y_train=y_train[:int(train_split*y_train.shape[0])]
    
        pred=model.predict(X_train)
        print('Training :- ')
        print(f'MSE : {np.mean((y_train-pred)**2)}')
        print(f'CV : {pearsonr(pred,y_train)[0]}')
    pred=model.predict(X_valid)
    print('Validation :- ')
    print(f'MSE : {np.mean((y_valid-pred)**2)}')
    print(f'CV : {pearsonr(pred,y_valid)[0]}')

def get_Xy_and_model_for_asset(df_train, asset_id):
    df = df_train[df_train["Asset_ID"] == asset_id]
   
    # TODO: Try different features here!
    df_proc = get_features(df)
    df_proc['y'] = df['Target']
    df_proc = df_proc.dropna(how="any")
    
    X = df_proc.drop("y", axis=1)
    y = df_proc["y"]
    
    X=np.array(X)
    
    input=Input(shape=X.shape[1:])

    x=Dense(64,activation=None,name='end')(input)
    
    x=Dense(X.shape[1])(x)
    model=Model(input,x)
    model.compile(loss='mae',optimizer=Adam(0.0001),metrics=['mae'])
    model.fit(X,X,epochs=12,batch_size=1,validation_split=0.2)
    Model(model.input,model.get_layer('end').output).save(f'model {asset_id}.h5')
    

In [ ]:
Xs = {}
ys = {}
models = {}

for asset_id, asset_name in zip(df_asset_details['Asset_ID'], df_asset_details['Asset_Name']):
    print(f"Training model for {asset_name:<16} (ID={asset_id:<2})")
    get_Xy_and_model_for_asset(df_train, asset_id)    
    